# 管线映射关系挖掘工具

本工具与 `shp-json-field-convert.ipynb` 功能类似，用于**从 JSON 数据和 Shapefile 中挖掘管线（conduit）的映射关系**。

## 主要功能

1. 读取管线 Shapefile 和 JSON 接口数据
2. 基于 `asset_id` 字段进行数据关联
3. 提取 systemtype（系统类型）属性
4. 生成序号（arraySequence）到 systemtype 的映射
5. 数据质量检查（匹配率、重复项警告）
6. 导出 CSV 和 JSON 格式的映射文件

## 与 shp-json-field-convert 的区别

- 本工具针对**管线（conduit/link）**数据
- shp-json-field-convert 针对**管井（node）**数据
- ID 字段名可能不同（`asset_id` vs `id`）

## 应用场景

- 管线数据与业务系统整合
- 系统类型映射表生成
- 数据完整性检查
- 多系统数据关联（污水、雨水、合流等）

> **依赖库**  
> - GeoPandas
> - Pandas
> - JSON（标准库）
> 
> 安装命令：  
> ```bash
> pip install geopandas pandas
> ```

In [23]:
# 安装：pip install geopandas pandas shapely pyproj fiona
import json
import pandas as pd
import geopandas as gpd
from pathlib import Path

# ======= 需按实际情况修改的参数 =======
SHP_PATH = r"F:\beipai\管井\0624\84版\links.shp"     # 你的shp路径
SHP_ID_COL = "asset_id"                              # shp里ID字段名
SHP_SYS_COL = "systemtype"                     # shp里systemtype字段名

JSON_PATH = r"F:\beipai\管井\管线管井序号编码\pipe.json"       # 接口JSON文件路径
OUTPUT_DIR = r"F:\beipai\管井\管线管井序号编码\links_out"                         


## 1. 配置参数

设置 Shapefile 路径、JSON 路径、字段名等。

In [19]:
gdf = gpd.read_file(SHP_PATH)  # 如遇编码乱码：gpd.read_file(SHP_PATH, encoding="gbk")

C:\Users\lishuhao\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: One or several characters couldn't be converted correctly from CP936 to UTF-8.  This warning will not be emitted anymore
  return ogr_read(


## 2. 读取 Shapefile

In [20]:
gdf

,id,asset_id,asset_uid,baseheight,bot_rcw,bot_rhw,bot_man,bot_n,branch_id,capacity,...,UT4,UT5,UT6,UT7,UT8,UT9,hotlinks,link_shape,gscc,geometry
0,B000016.1,B000016,None,0,1.5,0.0,25.0,0.014,NaN,0.722,...,污水三级,None,None,None,None,None,None,round,1500,"LINESTRING (116.52 39.975, 116.52 39.975)"
1,B000017.1,B000017,None,0,1.5,0.0,25.0,0.014,NaN,0.460,...,污水三级,None,None,None,None,None,None,round,1500,"LINESTRING (116.52 39.975, 116.52 39.974)"
2,B000018.1,B000018,None,0,1.5,0.0,25.0,0.014,NaN,1.443,...,污水三级,None,None,None,None,None,None,round,1500,"LINESTRING (116.52 39.974, 116.52 39.974)"
3,B000020.1,B000020,None,0,1.5,0.0,25.0,0.014,NaN,0.105,...,污水四级,None,None,None,None,None,None,round,1000,"LINESTRING (116.52 39.974, 116.52 39.974)"
4,B000197.1,B000233,None,0,1.5,NaN,25.0,0.014,NaN,0.073,...,污水三级,None,None,None,None,None,None,round,1000,"LINESTRING (116.6 39.961, 116.6 39.961)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245556,S009904.1,S000628,None,0,1.5,NaN,25.0,0.014,NaN,0.247,...,污水四级,None,None,None,None,None,None,round,1000,"LINESTRING (116.32 39.898, 116.32 39.898)"
245557,S022178.1,S026285,None,0,1.5,NaN,25.0,0.014,NaN,0.299,...,污水四级,None,None,None,None,None,None,round,1000,"LINESTRING (116.32 39.898, 116.32 39.898)"
245558,S022183.1,S026240,None,0,1.5,NaN,25.0,0.014,NaN,0.399,...,污水五级,None,None,None,None,None,None,round,1000,"LINESTRING (116.32 39.898, 116.32 39.898)"
245559,S033359.1,S033539,None,0,1.5,NaN,25.0,0.014,NaN,3.795,...,污水五级,雨水二级,None,None,None,None,None,rectangle,2200*1700,"LINESTRING (116.32 39.898, 116.32 39.898)"


In [21]:
gdf = gpd.read_file(SHP_PATH)                # 如中文乱码可加 encoding="gbk"
gdf[SHP_ID_COL] = gdf[SHP_ID_COL].astype(str)
shp_df = gdf[[SHP_ID_COL, SHP_SYS_COL]].copy()

C:\Users\lishuhao\AppData\Local\anaconda3\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: One or several characters couldn't be converted correctly from CP936 to UTF-8.  This warning will not be emitted anymore
  return ogr_read(


## 3. 读取 JSON 数据

In [24]:
# ======= 2) 读取 JSON（含 code/info/result 结构），提取 result 列表 =======
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

if not isinstance(data, dict) or "result" not in data:
    raise ValueError("JSON 顶层应为 dict 且包含 'result'。")

result = data["result"]
if not isinstance(result, list):
    raise ValueError("'result' 应为列表。")


## 4. 标准化字段名

In [25]:
# 标准化为 seq、id 两列（arraySequence -> seq, nodeId -> id）
json_df = pd.DataFrame(result)
required_cols = {"arraySequence", "conduitId"}
missing = required_cols - set(json_df.columns)
if missing:
    raise ValueError(f"result 中缺少字段: {missing}")

json_df = json_df.rename(columns={"arraySequence": "seq", "conduitId": "id"})
# 统一类型，避免 001 与 1 的不一致
json_df["id"] = json_df["id"].astype(str)
# seq 既可保持为整型，也可转字符串；这里保留整型以便排序
# json_df["seq"] = json_df["seq"].astype(str)

In [26]:
result

[{'conduitId': '沈家坟干渠_2', 'arraySequence': '0'},
 {'conduitId': '沈家坟干渠_3', 'arraySequence': '1'},
 {'conduitId': '沈家坟干渠_4', 'arraySequence': '2'},
 {'conduitId': '沈家坟干渠_5', 'arraySequence': '3'},
 {'conduitId': '沈家坟干渠_6', 'arraySequence': '4'},
 {'conduitId': '沈家坟干渠_7', 'arraySequence': '5'},
 {'conduitId': '沈家坟干渠_8', 'arraySequence': '6'},
 {'conduitId': '沈家坟干渠_9', 'arraySequence': '7'},
 {'conduitId': '沈家坟干渠_10', 'arraySequence': '8'},
 {'conduitId': '沈家坟干渠_11', 'arraySequence': '9'},
 {'conduitId': '沈家坟干渠_12', 'arraySequence': '10'},
 {'conduitId': '沈家坟干渠_13', 'arraySequence': '11'},
 {'conduitId': '沈家坟干渠_14', 'arraySequence': '12'},
 {'conduitId': '沈家坟干渠_15', 'arraySequence': '13'},
 {'conduitId': '沈家坟干渠_16', 'arraySequence': '14'},
 {'conduitId': '沈家坟干渠_17', 'arraySequence': '15'},
 {'conduitId': '沈家坟干渠_18', 'arraySequence': '16'},
 {'conduitId': '沈家坟干渠_19', 'arraySequence': '17'},
 {'conduitId': '沈家坟干渠_20', 'arraySequence': '18'},
 {'conduitId': '沈家坟干渠_21', 'arraySequence': '19'}

## 5. 数据关联

In [14]:
# ======= 3) 关联（以 JSON 为主，LEFT JOIN shp） =======
merged = json_df.merge(shp_df, left_on="id", right_on=SHP_ID_COL, how="left")
merged = merged.drop(columns=[SHP_ID_COL]).rename(columns={SHP_SYS_COL: "systemtype"})

## 6. 数据质量检查

In [27]:
# ======= 4) 质量检查 =======
not_matched = merged[merged["systemtype"].isna()]
print(f"[INFO] JSON 中序号总数：{len(json_df)}")
print(f"[INFO] 成功匹配到 shp 的条数：{len(merged) - len(not_matched)}")
print(f"[INFO] 未匹配到 shp 的条数：{len(not_matched)}")

# 同一 id 在 shp 是否重复（可能导致一个 seq 对应多个 systemtype）
dup_id = shp_df[shp_df.duplicated(subset=[SHP_ID_COL], keep=False)].sort_values(SHP_ID_COL)
if not dup_id.empty:
    print("[WARN] 下列 id 在 shp 中出现多次（示例前 10 行）：")
    print(dup_id.head(10))

[INFO] JSON 中序号总数：247810
[INFO] 成功匹配到 shp 的条数：233344
[INFO] 未匹配到 shp 的条数：11972
[WARN] 下列 id 在 shp 中出现多次（示例前 10 行）：
           asset_id systemtype
240627  ADDED_21004      storm
144816  ADDED_21004   combined
217772   AGDYS_0006      storm
217771   AGDYS_0006      storm
241331      B006367       foul
241311      B006367       foul
217755      B011463       foul
241032      B011463       foul
241033      B011463       foul
241068      Des_380      storm


## 7. 生成映射表（CSV）

In [18]:
seq2types = (merged.dropna(subset=["systemtype"])
                   .groupby("seq")["systemtype"]
                   .agg(lambda s: sorted(set(s)))
                   .reset_index())
seq2types_path = str(Path(OUTPUT_DIR, "seq_to_systemtypes.csv"))
seq2types.to_csv(seq2types_path, index=False, encoding="utf-8-sig")
print(f"[DONE] 序号→systemtype 映射：{seq2types_path}")


OSError: Cannot save file into a non-existent directory: 'F:\beipai\管井\管线管井序号编码\node_out'

## 8. 转换为 Python 字典

In [28]:
import json
from pathlib import Path


# 还是在你已有 seq2types 之后执行
seq2types = (
    merged.dropna(subset=["systemtype"])
          .groupby("seq")["systemtype"]
          .agg(lambda s: sorted(set(s)))
          .reset_index()
)

# 转为 Python dict：{seq: systemtype或[systemtype1, systemtype2,...]}
seq2types_dict = {}
for _, row in seq2types.iterrows():
    seq = int(row["seq"]) if str(row["seq"]).isdigit() else str(row["seq"])
    systems = row["systemtype"]
    # 如果只有一个元素，就直接取字符串而不是列表
    if isinstance(systems, (list, tuple)) and len(systems) == 1:
        seq2types_dict[seq] = systems[0]
    else:
        seq2types_dict[seq] = systems

## 9. 导出为 JSON 文件

In [29]:
# 导出为 JSON 文件
seq2types_path = Path(OUTPUT_DIR) / "links.json"
with open(seq2types_path, "w", encoding="utf-8") as f:
    json.dump(seq2types_dict, f, ensure_ascii=False, indent=2)

print(f"[DONE] 已导出 JSON 文件：{seq2types_path}")

[DONE] 已导出 JSON 文件：F:\beipai\管井\管线管井序号编码\links_out\links.json
